In [1]:
from busy_beaver_blaze import Visualizer, BB5_CHAMP, BB6_CONTENDER  # noqa: F401


In [2]:
import itertools
from busy_beaver_blaze import Machine

machine = Machine("1RB0LD_1LC1RD_1LA1LC_1RZ1RE_1RA0RB")
step_limit = 100_000_000

print(f"Created a machine with {machine.program.state_count} states")

# Step 0 is the initial state, so we can run one less than the limit
steps_run = 1 + sum(1 for _ in itertools.islice(machine, step_limit - 1))

print(f"Machine ran for a total: {steps_run:,} steps including initial state")
print(f"Number of ones on tape: {machine.count_ones():,}")

Created a machine with 5 states
Machine ran for a total: 23,554,764 steps including initial state
Number of ones on tape: 4,097


In [3]:
from time import perf_counter
from busy_beaver_blaze import run_machine_steps

rust_repeat_count = 100
python_repeat_count = 1
step_limit = 100_000_000

rust_total = 0.0
rust_steps = None
rust_nonzeros = None
for _ in range(rust_repeat_count):
    start = perf_counter()
    current_steps, current_nonzeros = run_machine_steps(
        "1RB0LD_1LC1RD_1LA1LC_1RZ1RE_1RA0RB",
        step_limit,
    )
    rust_total += perf_counter() - start
    if rust_steps is None:
        rust_steps = current_steps
        rust_nonzeros = current_nonzeros
    else:
        assert current_steps == rust_steps
        assert current_nonzeros == rust_nonzeros

rust_average = rust_total / rust_repeat_count
rust_message = (
    f"Rust helper ran {rust_steps:,} steps with {rust_nonzeros:,} ones "
    f"(avg {rust_average:.3f}s per run over {rust_repeat_count} runs)"
)
print(rust_message)

python_total = 0.0
python_steps = None
python_nonzeros = None
for _ in range(python_repeat_count):
    start = perf_counter()
    current_steps, current_nonzeros = run_machine_steps(
        "1RB0LD_1LC1RD_1LA1LC_1RZ1RE_1RA0RB",
        step_limit,
        force_python_only=True,
    )
    python_total += perf_counter() - start
    if python_steps is None:
        python_steps = current_steps
        python_nonzeros = current_nonzeros
    else:
        assert current_steps == python_steps
        assert current_nonzeros == python_nonzeros

python_average = python_total / python_repeat_count
python_message = (
    f"Python-only helper ran {python_steps:,} steps with {python_nonzeros:,} ones "
    f"(avg {python_average:.3f}s per run over {python_repeat_count} runs)"
)
print(python_message)

assert rust_steps == python_steps
assert rust_nonzeros == python_nonzeros


Rust helper ran 23,554,764 steps with 4,097 ones (avg 0.101s per run over 100 runs)
Python-only helper ran 23,554,764 steps with 4,097 ones (avg 10.170s per run over 1 runs)
